In [ ]:
# !pip install opentelemetry-api
# !pip install opentelemetry-sdk

In [ ]:
from pathlib import Path
import sys

# Add base dir to path to be able to access test folder.
base_dir = Path().cwd().parent.parent.resolve()
if str(base_dir) not in sys.path:
    print(f"Adding {base_dir} to sys.path")
    sys.path.append(str(base_dir))

In [ ]:
from opentelemetry import trace
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor
from opentelemetry.sdk.trace.export import ConsoleSpanExporter

SERVICE_NAME = "trulens"


def init():
    # Use Resource.create() instead of constructor directly
    resource = Resource.create({"service.name": SERVICE_NAME})

    trace.set_tracer_provider(TracerProvider(resource=resource))
    trace.get_tracer_provider().add_span_processor(
        BatchSpanProcessor(ConsoleSpanExporter())
    )


init()

In [ ]:
from typing import Callable

from trulens.apps.custom import instrument

SERVICE_NAME = "trulens"


def decorator(func: Callable):
    tracer = trace.get_tracer(SERVICE_NAME)

    def wrapper(*args, **kwargs):
        print("start wrap")

        with tracer.start_as_current_span("custom"):
            result = func(*args, **kwargs)
            span = trace.get_current_span()
            print("---span---")
            print(span.get_span_context())
            span.set_attribute("result", result)
            span.set_status(trace.Status(trace.StatusCode.OK))
            return result

    return wrapper

In [ ]:
from examples.dev.dummy_app.dummy import Dummy


class TestApp(Dummy):
    def __init__(self):
        super().__init__()

    @decorator
    @instrument
    def respond_to_query(self, query: str) -> str:
        return f"answer: {self.nested(query)}"

    @decorator
    @instrument
    def nested(self, query: str) -> str:
        return f"nested: {query}"

In [ ]:
from trulens.core.session import TruSession

SERVICE_NAME = "trulens"

In [ ]:
session = TruSession()